### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [6]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
grouped = reviews.groupby('movie_id')
review_stats = pd.DataFrame(index=movies.movie_id, columns=[
    'mean_rating', 'num_ratings', 'most_recent_rating'])
review_stats.mean_rating = grouped.rating.mean()
review_stats.num_ratings = grouped.count()
review_stats.most_recent_rating = grouped.date.max()
review_stats.head()

,mean_rating,num_ratings,most_recent_rating
movie_id,,,
8,5.0,1,2014-04-08 18:20:11
10,10.0,1,2014-10-09 18:15:53
12,10.0,1,2015-08-10 23:16:19
25,8.0,1,2017-02-27 10:04:59
91,6.0,1,2013-11-23 18:59:55


In [44]:
sorted_movies = review_stats.sort_values(
    by=['mean_rating', 'num_ratings', 'most_recent_rating'],
    ascending=[False, False, False]
)
sorted_movies = sorted_movies[sorted_movies.num_ratings >= 5]
sorted_movies.head(35)

,mean_rating,num_ratings,most_recent_rating
movie_id,,,
4921860,10.000000,48,2016-08-14 17:16:50
5262972,10.000000,28,2016-01-08 00:44:43
5688932,10.000000,14,2018-06-17 01:44:48
2737018,10.000000,10,2015-05-10 22:56:01
2560840,10.000000,6,2016-01-23 00:30:44
2219210,10.000000,6,2013-07-22 23:30:52
4448444,10.000000,5,2017-09-09 13:51:48
5131914,10.000000,5,2016-04-04 05:52:22
2059318,10.000000,5,2014-02-13 15:48:31


In [45]:
top_movies = sorted_movies.reset_index().iloc[:20]
top_movies.merge(movies, on='movie_id').movie.tolist()

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']

In [54]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    grouped = reviews.groupby('movie_id')
    review_stats = pd.DataFrame(index=movies.movie_id, columns=[
        'mean_rating', 'num_ratings', 'most_recent_rating'])
    review_stats.mean_rating = grouped.rating.mean()
    review_stats.num_ratings = grouped.count()
    review_stats.most_recent_rating = grouped.date.max()
    sorted_movies = review_stats.sort_values(
        by=['mean_rating', 'num_ratings', 'most_recent_rating'],
        ascending=[False, False, False]
    )
    sorted_movies = sorted_movies[sorted_movies.num_ratings >= 5]
    top_movies = sorted_movies.reset_index().iloc[:n_top]
    top_movies = top_movies.merge(movies, on='movie_id').movie.tolist()
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [55]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35)



In [61]:
### You Should Not Need To Modify Anything In This Cell
%time ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

CPU times: user 1min 10s, sys: 1.05 s, total: 1min 11s
Wall time: 1min 11s
If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [92]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [63]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [93]:
years=['2015', '2016', '2017', '2018']
genres=['History']

In [97]:
def filter_year_genre(movies, years, genres):
    in_years = movies.date.isin(years)
    sub_movies = movies[genres]
    in_genre = sub_movies.sum(axis=1) > 0
    subset = movies[in_years & in_genre]
    
    return subset

In [98]:
in_years = movies.date.isin(years)
sub_movies = movies[genres]
in_genre = sub_movies.sum(axis=1) > 0
subset = movies[in_years & in_genre]
subset.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
13156,420293,The Stanford Prison Experiment (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14259,490215,Silence (2016),Drama|History,2016,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
15853,1083452,Eddie the Eagle (2016),Biography|Drama|History,2016,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16103,1135989,A Tale of Love and Darkness (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16753,1254322,The Girl King (2015),Biography|Drama|History,2015,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
def popular_recommendations(user_id, n_top, years, genres):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    subset = filter_year_genre(movies, years, genres)
    grouped = reviews.groupby('movie_id')
    review_stats = pd.DataFrame(index=subset.movie_id, columns=[
        'mean_rating', 'num_ratings', 'most_recent_rating'])
    review_stats.mean_rating = grouped.rating.mean()
    review_stats.num_ratings = grouped.count()
    review_stats.most_recent_rating = grouped.date.max()
    sorted_movies = review_stats.sort_values(
        by=['mean_rating', 'num_ratings', 'most_recent_rating'],
        ascending=[False, False, False]
    )
    sorted_movies = sorted_movies[sorted_movies.num_ratings >= 5]
    top_movies = sorted_movies.reset_index().iloc[:n_top]
    top_movies = top_movies.merge(subset, on='movie_id').movie.tolist()
    return top_movies # a list of the n_top movies as recommended

In [102]:
recom_1 = popular_recommendations('1', 20, 
                                  years=['2015', '2016', '2017', '2018'], 
                                  genres=['History'])
sol_1 = t.popular_recs_filtered(
    '1', 20, ranked_movies, 
    years=['2015', '2016', '2017', '2018'], 
    genres=['History'])

assert sol_1 == recom_1

print('Well done, myself!')

Well done, myself!
